### Problem analysis

In [1]:
import pandas as pd
import cProfile

In [ ]:
df = pd.read_csv("../notes/protocol_default.csv", comment="#", delim_whitespace=True)
df["Usertime"] = df["Usertime"].round(decimals=0)
df = df.sort_values("Usertime", ascending=False)

df

/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_39734/1993705285.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("../notes/protocol_default.csv", comment="#", delim_whitespace=True)


,Problem,Status,Usertime,Failure,Version,Preprocessingtime,Backward_subsumed,Factors_computed,Forward_subsumed,Initial_clauses,Processed_clauses,Resolvents_computed,Systemtime,Tautologies_deleted,Totaltime
13131,SWV194+1.p,T,99.852,success,1.2,-,4,1,1960,357,2166,98975,0.371,2,100.223
5358,LAT261-1.p,T,99.759,success,1.2,-,12,5,5673,3038,3687,61459,0.138,72,99.897
11919,SWB011+1.p,T,99.539,success,1.2,-,25,198,4675,1281,3675,43267,0.120,58,99.659
5094,LAT005-2.p,T,98.176,success,1.2,-,55,0,4391,31,3966,9397,0.032,339,98.208
2021,FLD021-3.p,T,97.506,success,1.2,-,6,8,96,30,1970,277716,0.371,10,97.877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5722,LAT362+3.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5723,LAT362+4.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5724,LAT363+1.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-
5725,LAT363+2.p,F,-,unknown,1.2,-,-,-,-,-,-,-,-,-,-


In [ ]:
import re

patterns = {
    "spc": r'(?<=% SPC      : )(\w+)',
    "n_formulae": r'(?<=formulae    :  )(\d+)',
    "n_atoms": r'(?<=atoms       :  )(\d+)',
}
for problem in df["Problem"]:
    with open(f"TPTP-v9.0.0/Problems/{problem[:3]}/{problem}") as file:
        problem_config: dict
        for identifier, pattern in patterns.items():
            problem_config[identifier] = re.search(pattern, file.read()).group()





### Profiling

In [18]:
import pyres_fof
import cProfile
import re

opts = [
    ("-s", ""),
    ("-r", "12"),
]
args = [
    "EXAMPLES/PUZ002-1.p"
]
with cProfile.Profile() as pr:
    pyres_fof.main(from_notebook=True, notebook_opts=opts, notebook_args=args)
    df_stats = pd.DataFrame(
        pr.getstats(),
        columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"]
    )
df_stats["func"] = df_stats["func"].astype(str)
df_stats = df_stats[ df_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]

func_names = []
lines = []
for value in df_stats["func"]:
    func_name = re.search(r'(?<=code object )(\w+)', value)
    line = re.search(r'(?<=line )(\d+)', value)
    func_names.append(func_name.group() if func_name is not None else None)
    lines.append(int(line.group()) if line is not None else None)
df_stats["func_name"] = func_names
df_stats["line"] = lines

df_stats = (
        df_stats[["func_name", "line", "ncalls", "ccalls", "tottime", "cumtime", "func", "callers"]]
        .sort_values("ncalls", ascending=False)
)

df_stats.to_csv("profiling_runs.csv")
df_stats

% SZS status Unsatisfiable

% Initial clauses    : 12
% Processed clauses  : 22
% Factors computed   : 0
% Resolvents computed: 22
% Tautologies deleted: 0
% Forward subsumed   : 0
% Backward subsumed  : 0
% -------- CPU Time ---------
% User time          : 177.018 s
% System time        : 0.797 s
% Total time         : 177.816 s


/var/folders/pb/8wrfvtz5593cjrgr_g89r70m0000gn/T/ipykernel_39734/4271083216.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_stats = df_stats[ df_stats["func"].str.contains(r"PyRes/(\w+)\.py") ]


,func_name,line,ncalls,ccalls,tottime,cumtime,func,callers
207,termIsVar,73,5671,0,1.326936e-03,1.326936e-03,"<code object termIsVar at 0x10ad30a30, file ""/...",None
216,termIsCompound,81,3677,0,1.879591e-03,1.054408e-03,"<code object termIsCompound at 0x10ad384e0, fi...","[(<code object termIsVar at 0x10ad30a30, file ..."
206,__len__,90,1957,0,6.064870e-04,4.674740e-04,"<code object __len__ at 0x10ad30930, file ""/Us...","[(<built-in method builtins.len>, 1957, 0, 0.0..."
208,termFunc,89,1812,0,1.420533e-03,5.242790e-04,"<code object termFunc at 0x10ad30b30, file ""/U...",[(<method '__exit__' of 'sqlite3.Connection' o...
200,getLiteral,115,1552,0,1.311363e-03,4.255870e-04,"<code object getLiteral at 0x10ad28c90, file ""...","[(<built-in method builtins.len>, 1552, 0, 0.0..."
...,...,...,...,...,...,...,...,...
210,get_all_nodes,65,1,0,1.875000e-06,1.875000e-06,"<code object get_all_nodes at 0x10ad31930, fil...",None
213,isPred,95,1,0,1.660000e-07,1.660000e-07,"<code object isPred at 0x10ad38120, file ""/Use...",None
225,clausify,147,1,0,1.595800e-05,7.916000e-06,"<code object clausify at 0x10ad68030, file ""/U...","[(<code object __init__ at 0x10ab1b690, file ""..."
226,saturate,201,1,0,2.468350e-02,1.950100e-05,"<code object saturate at 0x10ad74570, file ""/U...","[(<code object __len__ at 0x10aae3d30, file ""/..."
